# Text Classification Model for Sentiment Labelled Sentences Using TensorFlow Take 3
### David Lowe
### January 19, 2021

Template Credit: Adapted from a template made available by Dr. Jason Brownlee of Machine Learning Mastery. [https://machinelearningmastery.com/]

SUMMARY: This project aims to construct a text classification model using a neural network and document the end-to-end steps using a template. The Sentiment Labelled Sentences dataset is a binary classification situation where we attempt to predict one of the two possible outcomes.

INTRODUCTION: This dataset was created for the research paper 'From Group to Individual Labels using Deep Features,' Kotzias et al., KDD 2015. The paper researchers randomly selected 500 positive and 500 negative sentences from a larger dataset of reviews for each website. The researcher also attempted to choose sentences with a positive or negative connotation as the goal was to avoid selecting neutral sentences.

From iteration Take1, we deployed a bag-of-words model to classify the Amazon dataset's review comments. We also applied various sequence-to-matrix modes to evaluate the model's performance.

From iteration Take2, we deployed a word-embedding model to classify the Amazon dataset's review comments. We also compared the result with the bag-of-word model from the previous iteration.

In this Take3 iteration, we will deploy a bag-of-words model to classify the IMDB dataset's review comments. We will also apply various sequence-to-matrix modes to evaluate the model's performance.

ANALYSIS: From iteration Take1, the bag-of-words model's performance achieved an average accuracy score of 77.31% after 25 epochs with ten iterations of cross-validation. Furthermore, the final model processed the test dataset with an accuracy measurement of 71.00%.

From iteration Take2, the word-embedding model's performance achieved an average accuracy score of 73.25% after 25 epochs with ten iterations of cross-validation. Furthermore, the final model processed the test dataset with an accuracy measurement of 67.00%.

In this Take3 iteration, the bag-of-words model's performance achieved an average accuracy score of 77.26% after 25 epochs with ten iterations of cross-validation. Furthermore, the final model processed the test dataset with an accuracy measurement of 68.66%.

CONCLUSION: In this modeling iteration, the bag-of-words TensorFlow model appeared to be suitable for modeling this dataset. We should consider experimenting with TensorFlow for further modeling.

Dataset Used: Sentiment Labelled Sentences

Dataset ML Model: Binary class text classification with text-oriented features

Dataset Reference: https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences

A deep-learning text classification project generally can be broken down into five major tasks:

1. Prepare Environment
2. Load and Prepare Text Data
3. Define and Train Models
4. Evaluate and Optimize Models
5. Finalize Model and Make Predictions

# Task 1 - Prepare Environment

In [1]:
# # Install the packages to support accessing environment variable and SQL databases
# !pip install python-dotenv PyMySQL boto3

In [2]:
# # Retrieve GPU configuration information from Colab
# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#     print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
#     print('and then re-execute this cell.')
# else:
#     print(gpu_info)

In [3]:
# # Retrieve memory configuration information from Colab
# from psutil import virtual_memory
# ram_gb = virtual_memory().total / 1e9
# print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

# if ram_gb < 20:
#     print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
#     print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
#     print('re-execute this cell.')
# else:
#     print('You are using a high-RAM runtime!')

In [4]:
# Retrieve CPU information from the system
ncpu = !nproc
print("The number of available CPUs is:", ncpu[0])

The number of available CPUs is: 4


## 1.a) Load libraries and modules

In [5]:
# Set the random seed number for reproducible results
seedNum = 1

In [6]:
# Load libraries and packages
import random
random.seed(seedNum)
import numpy as np
np.random.seed(seedNum)
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import sys
import boto3
import shutil
import string
import nltk
from nltk.corpus import stopwords
from collections import Counter
from datetime import datetime
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import tensorflow as tf
tf.random.set_seed(seedNum)
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [7]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/pythonml/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/pythonml/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/pythonml/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/pythonml/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/pythonml/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/pythonml/nltk_data...
[nltk_data]    |   Package movie_reviews is a

True

## 1.b) Set up the controlling parameters and functions

In [8]:
# Begin the timer for the script processing
startTimeScript = datetime.now()

# Set up the number of CPU cores available for multi-thread processing
n_jobs = 1

# Set up the flag to stop sending progress emails (setting to True will send status emails!)
notifyStatus = False

# Set the verbose level for program execution output
verbose = False

# Set Pandas options
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 140)

# Set the percentage sizes for splitting the dataset
TEST_SET_SIZE = 0.2
VAL_SET_SIZE = 0.25

# Set the number of folds for cross validation
N_FOLDS = 5
N_ITERATIONS = 2

# Set various default modeling parameters
DEFAULT_LOSS = 'binary_crossentropy'
DEFAULT_METRICS = ['accuracy']
DEFAULT_OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=0.001)
DEFAULT_INITIALIZER = tf.keras.initializers.GlorotUniform(seed=seedNum)
EPOCH_NUMBER = 25
BATCH_SIZE = 16

# Define the labels to use for graphing the data
train_metric = "accuracy"
validation_metric = "val_accuracy"
train_loss = "loss"
validation_loss = "val_loss"

# Check the number of GPUs accessible through TensorFlow
print('Num GPUs Available:', len(tf.config.list_physical_devices('GPU')))

# Print out the TensorFlow version for confirmation
print('TensorFlow version:', tf.__version__)

Num GPUs Available: 0
TensorFlow version: 2.3.1


In [9]:
# Set up the parent directory location for loading the dotenv files
# from google.colab import drive
# drive.mount('/content/gdrive')
# gdrivePrefix = '/content/gdrive/My Drive/Colab_Downloads/'
# env_path = '/content/gdrive/My Drive/Colab Notebooks/'
# dotenv_path = env_path + "python_script.env"
# load_dotenv(dotenv_path=dotenv_path)

# Set up the dotenv file for retrieving environment variables
# env_path = "/Users/david/PycharmProjects/"
# dotenv_path = env_path + "python_script.env"
# load_dotenv(dotenv_path=dotenv_path)

In [10]:
# Set up the email notification function
def status_notify(msg_text):
    access_key = os.environ.get('SNS_ACCESS_KEY')
    secret_key = os.environ.get('SNS_SECRET_KEY')
    aws_region = os.environ.get('SNS_AWS_REGION')
    topic_arn = os.environ.get('SNS_TOPIC_ARN')
    if (access_key is None) or (secret_key is None) or (aws_region is None):
        sys.exit("Incomplete notification setup info. Script Processing Aborted!!!")
    sns = boto3.client('sns', aws_access_key_id=access_key, aws_secret_access_key=secret_key, region_name=aws_region)
    response = sns.publish(TopicArn=topic_arn, Message=msg_text)
    if response['ResponseMetadata']['HTTPStatusCode'] != 200 :
        print('Status notification not OK with HTTP status code:', response['ResponseMetadata']['HTTPStatusCode'])

In [11]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 1 - Prepare Environment has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

In [12]:
# Reset the random number generators
def reset_random(x):
    random.seed(x)
    np.random.seed(x)
    tf.random.set_seed(x)

In [13]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 1 - Prepare Environment completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

# Task 2 - Load and Prepare Text Data

In [14]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 2 - Load and Prepare Text Data has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

## 2.a) Download Text Data Archive

In [15]:
dataset_path = 'https://dainesanalytics.com/datasets/ucirvine-sentiment-labelled-sentences/imdb_labelled.txt'
colNames = ['comment','targetVar']
Xy_original = pd.read_csv(dataset_path, names=colNames, sep='\t', header=None, index_col=False)

# Take a peek at the dataframe after import
print(Xy_original.head(10))

                                             comment  targetVar
0  A very, very, very slow-moving, aimless movie ...          0
1  Not sure who was more lost - the flat characte...          0
2  Attempting artiness with black & white and cle...          0
3       Very little music or anything to speak of.            0
4  The best scene in the movie was when Gerardo i...          1
5  The rest of the movie lacks art, charm, meanin...          0
6                                Wasted two hours.            0
7  Saw the movie today and thought it was a good ...          1
8                               A bit predictable.            0
9  Loved the casting of Jimmy Buffet as the scien...          1


## 2.b) Splitting Data for Training and Validation

In [16]:
X_original = Xy_original.iloc[:,0]
y_original = Xy_original.iloc[:,1]
print("Xy_original.shape: {} X_original.shape: {} y_original.shape: {}".format(Xy_original.shape, X_original.shape, y_original.shape))
print(X_original.head())

Xy_original.shape: (748, 2) X_original.shape: (748,) y_original.shape: (748,)
0    A very, very, very slow-moving, aimless movie ...
1    Not sure who was more lost - the flat characte...
2    Attempting artiness with black & white and cle...
3         Very little music or anything to speak of.  
4    The best scene in the movie was when Gerardo i...
Name: comment, dtype: object


In [17]:
# Split the data further into training and test datasets
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X_original, y_original, test_size=TEST_SET_SIZE, stratify=y_original, random_state=seedNum)
print("X_train_df.shape: {} y_train_df.shape: {}".format(X_train_df.shape, y_train_df.shape))
print("X_test_df.shape: {} y_test_df.shape: {}".format(X_test_df.shape, y_test_df.shape))

X_train_df.shape: (598,) y_train_df.shape: (598,)
X_test_df.shape: (150,) y_test_df.shape: (150,)


## 2.c) Load Document and Build Vocabulary

In [18]:
# turn a doc into clean tokens
def clean_sentence(sentence):
    # split into tokens by white space
    tokens = sentence.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

In [19]:
# load sentences and add to vocab
def add_sentence_to_vocab(sentence, vocab):
    # clean doc
    tokens = clean_sentence(sentence)
    # update counts
    vocab.update(tokens)

In [20]:
# load all docs in a directory
def build_vocabulary(comments, vocab):
    # walk through all comments in the dataframe
    for i in range(len(comments)):
        # add comments to vocab
        sentence = comments.iloc[i]
        add_sentence_to_vocab(sentence, vocab)
        if verbose : print('Processing comment:', sentence)
        if ((i+1) % 100) == 0 : print(i+1, 'comments processed so far.')
    print('Total number of comments loaded into the vocabulary:', i+1, '\n')

In [21]:
# define vocab
vocab = Counter()
# add all docs to vocab
build_vocabulary(X_train_df, vocab)
# print the size of the vocab
print('The total number of words in the vocabulary:', len(vocab))
# print the top words in the vocab
top_words = 50
print('The top', top_words, 'words in the vocabulary:\n', vocab.most_common(top_words))

100 comments processed so far.
200 comments processed so far.
300 comments processed so far.
400 comments processed so far.
500 comments processed so far.
Total number of comments loaded into the vocabulary: 598 

The total number of words in the vocabulary: 2742
The top 50 words in the vocabulary:
 [('The', 146), ('movie', 143), ('film', 129), ('This', 62), ('one', 58), ('bad', 47), ('good', 47), ('It', 44), ('time', 38), ('like', 35), ('acting', 31), ('even', 30), ('see', 29), ('really', 29), ('characters', 28), ('great', 26), ('Its', 25), ('films', 23), ('movies', 22), ('made', 22), ('well', 21), ('story', 21), ('But', 20), ('seen', 20), ('ever', 20), ('make', 19), ('plot', 18), ('think', 18), ('much', 18), ('real', 18), ('best', 18), ('love', 18), ('scenes', 18), ('could', 18), ('also', 17), ('dont', 17), ('didnt', 17), ('character', 17), ('would', 17), ('watching', 17), ('script', 16), ('way', 16), ('work', 15), ('If', 15), ('every', 14), ('wonderful', 14), ('There', 14), ('anyone

In [22]:
# keep tokens with a min occurrence
min_occurane = 2
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print('The number of words with the minimum appearance:', len(tokens))

The number of words with the minimum appearance: 884


In [23]:
# save list to file
def save_list(lines, filename):
    # convert lines to a single blob of text
    data = '\n'.join(lines)
    # open file
    file = open(filename, 'w')
    # write text
    file.write(data)
    # close file
    file.close()

# save tokens to a vocabulary file
save_list(tokens, 'vocabulary.txt')

In [24]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [25]:
# load the vocabulary
vocab_filename = 'vocabulary.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
print('Number of tokens in the vocabulary:', len(vocab))

Number of tokens in the vocabulary: 884


## 2.d) Create Tokenizer and Encode the Input Text

In [26]:
# load doc, clean and return line of tokens
def comment_to_line(sentence, vocab):
    # clean sentence
    tokens = clean_sentence(sentence)
    # filter by vocab
    tokens = [w for w in tokens if w in vocab]
    line = ' '.join(tokens)
    return line

In [27]:
# load all docs in a directory
def process_comments_to_lines(comments, vocab):
    lines = list()
    # walk through all comments in the dataframe
    for i in range(len(comments)):
        # load and clean the comments
        sentence = comments.iloc[i]
        line = comment_to_line(sentence, vocab)
        # add to list
        lines.append(line)
    return lines

In [28]:
# # Load all training cases
training_docs = process_comments_to_lines(X_train_df, vocab)

In [29]:
# # load all test/validation cases
testing_docs = process_comments_to_lines(X_test_df, vocab)

In [30]:
# prepare bag of words encoding of docs
def encode_data(train_docs, val_docs, mode='binary'):
    # create the tokenizer
    tokenizer = Tokenizer()
    # fit the tokenizer on the documents
    tokenizer.fit_on_texts(train_docs)
    # encode training data set
    train_encoded = tokenizer.texts_to_matrix(train_docs, mode=mode)
    # encode validation data set
    val_encoded = tokenizer.texts_to_matrix(val_docs, mode=mode)
    return train_encoded, val_encoded

In [31]:
# encode training and validation datasets
X_train, X_test = encode_data(training_docs, testing_docs)
print('The shape of the encoded training dataset:', X_train.shape)
print('The shape of the encoded validation dataset:', X_test.shape)

The shape of the encoded training dataset: (598, 849)
The shape of the encoded validation dataset: (150, 849)


In [32]:
y_train = y_train_df.values.ravel()
y_test = y_test_df.values.ravel()
print('The shape of the encoded test classes:', y_train.shape)
print('The shape of the encoded test classes:', y_test.shape)

The shape of the encoded test classes: (598,)
The shape of the encoded test classes: (150,)


In [33]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 2 - Load and Prepare Text Data completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

# Task 3 - Define and Train Models

In [34]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 3 - Define and Train Models has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

In [35]:
# Define the default numbers of input/output for modeling
num_inputs = X_train.shape[1]
num_outputs = 1

In [36]:
# Define the baseline model for benchmarking
def create_nn_model(n_inputs=num_inputs, n_outputs=num_outputs, layer1_nodes=50, layer1_dropout=0, opt_param=DEFAULT_OPTIMIZER, init_param=DEFAULT_INITIALIZER, loss_param=DEFAULT_LOSS, metrics_param=DEFAULT_METRICS):
    nn_model = keras.Sequential([
        keras.layers.Dense(layer1_nodes, input_shape=(n_inputs,), activation='relu', kernel_initializer=init_param),
#         keras.layers.Dropout(layer1_dropout),
        keras.layers.Dense(n_outputs, activation='sigmoid', kernel_initializer=init_param)
    ])
    nn_model.compile(loss=loss_param, optimizer=opt_param, metrics=metrics_param)
    return nn_model

In [37]:
# Initialize the default model and get a baseline result
startTimeModule = datetime.now()
results = list()
iteration = 0
cv = RepeatedKFold(n_splits=N_FOLDS, n_repeats=N_ITERATIONS, random_state=seedNum)
for train_ix, val_ix in cv.split(X_train):
    feature_train, feature_validation = X_train[train_ix], X_train[val_ix]
    target_train, target_validation = y_train[train_ix], y_train[val_ix]
    reset_random(seedNum)
    baseline_model = create_nn_model()
    baseline_model.fit(feature_train, target_train, epochs=EPOCH_NUMBER, batch_size=BATCH_SIZE, verbose=0)
    model_metric = baseline_model.evaluate(feature_validation, target_validation, verbose=0)[1]
    iteration = iteration + 1
    print('Accuracy measurement from iteration %d >>> %.2f%%' % (iteration, model_metric*100))
    results.append(model_metric)
validation_score = np.mean(results)
validation_variance = np.std(results)
print('Average model accuracy from all iterations: %.2f%% (%.2f%%)' % (validation_score*100, validation_variance*100))
print('Total time for model fitting and cross validating:', (datetime.now() - startTimeModule))

Accuracy measurement from iteration 1 >>> 76.67%
Accuracy measurement from iteration 2 >>> 78.33%
Accuracy measurement from iteration 3 >>> 74.17%
Accuracy measurement from iteration 4 >>> 81.51%
Accuracy measurement from iteration 5 >>> 75.63%
Accuracy measurement from iteration 6 >>> 76.67%
Accuracy measurement from iteration 7 >>> 73.33%
Accuracy measurement from iteration 8 >>> 71.67%
Accuracy measurement from iteration 9 >>> 76.47%
Accuracy measurement from iteration 10 >>> 82.35%
Average model accuracy from all iterations: 76.68% (3.19%)
Total time for model fitting and cross validating: 0:00:25.845940


In [38]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 3 - Define and Train Models completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

# Task 4 - Evaluate and Optimize Models

In [39]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 4 - Evaluate and Optimize Models has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

## 4.a) Alternate Model One

In [40]:
# encode training and validation datasets
X_train, X_test = encode_data(training_docs, testing_docs, 'count')
print('The shape of the encoded training dataset:', X_train.shape)
print('The shape of the encoded validation dataset:', X_test.shape)

The shape of the encoded training dataset: (598, 849)
The shape of the encoded validation dataset: (150, 849)


In [41]:
# Initialize the default model and get a baseline result
startTimeModule = datetime.now()
results = list()
iteration = 0
cv = RepeatedKFold(n_splits=N_FOLDS, n_repeats=N_ITERATIONS, random_state=seedNum)
for train_ix, val_ix in cv.split(X_train):
    feature_train, feature_validation = X_train[train_ix], X_train[val_ix]
    target_train, target_validation = y_train[train_ix], y_train[val_ix]
    reset_random(seedNum)
    alternate_model_1 = create_nn_model()
    alternate_model_1.fit(feature_train, target_train, epochs=EPOCH_NUMBER, batch_size=BATCH_SIZE, verbose=0)
    model_metric = alternate_model_1.evaluate(feature_validation, target_validation, verbose=0)[1]
    iteration = iteration + 1
    print('Accuracy measurement from iteration %d >>> %.2f%%' % (iteration, model_metric*100))
    results.append(model_metric)
validation_score = np.mean(results)
validation_variance = np.std(results)
print('Average model accuracy from all iterations: %.2f%% (%.2f%%)' % (validation_score*100, validation_variance*100))
print('Total time for model fitting and cross validating:', (datetime.now() - startTimeModule))

Accuracy measurement from iteration 1 >>> 74.17%
Accuracy measurement from iteration 2 >>> 78.33%
Accuracy measurement from iteration 3 >>> 73.33%
Accuracy measurement from iteration 4 >>> 81.51%
Accuracy measurement from iteration 5 >>> 75.63%
Accuracy measurement from iteration 6 >>> 75.83%
Accuracy measurement from iteration 7 >>> 74.17%
Accuracy measurement from iteration 8 >>> 74.17%
Accuracy measurement from iteration 9 >>> 75.63%
Accuracy measurement from iteration 10 >>> 79.83%
Average model accuracy from all iterations: 76.26% (2.59%)
Total time for model fitting and cross validating: 0:00:24.744888


## 4.a) Alternate Model Two

In [42]:
# encode training and validation datasets
X_train, X_test = encode_data(training_docs, testing_docs, 'tfidf')
print('The shape of the encoded training dataset:', X_train.shape)
print('The shape of the encoded validation dataset:', X_test.shape)

The shape of the encoded training dataset: (598, 849)
The shape of the encoded validation dataset: (150, 849)


In [43]:
# Initialize the default model and get a baseline result
startTimeModule = datetime.now()
results = list()
iteration = 0
cv = RepeatedKFold(n_splits=N_FOLDS, n_repeats=N_ITERATIONS, random_state=seedNum)
for train_ix, val_ix in cv.split(X_train):
    feature_train, feature_validation = X_train[train_ix], X_train[val_ix]
    target_train, target_validation = y_train[train_ix], y_train[val_ix]
    reset_random(seedNum)
    alternate_model_2 = create_nn_model()
    alternate_model_2.fit(feature_train, target_train, epochs=EPOCH_NUMBER, batch_size=BATCH_SIZE, verbose=0)
    model_metric = alternate_model_2.evaluate(feature_validation, target_validation, verbose=0)[1]
    iteration = iteration + 1
    print('Accuracy measurement from iteration %d >>> %.2f%%' % (iteration, model_metric*100))
    results.append(model_metric)
validation_score = np.mean(results)
validation_variance = np.std(results)
print('Average model accuracy from all iterations: %.2f%% (%.2f%%)' % (validation_score*100, validation_variance*100))
print('Total time for model fitting and cross validating:', (datetime.now() - startTimeModule))

Accuracy measurement from iteration 1 >>> 77.50%
Accuracy measurement from iteration 2 >>> 79.17%
Accuracy measurement from iteration 3 >>> 73.33%
Accuracy measurement from iteration 4 >>> 82.35%
Accuracy measurement from iteration 5 >>> 77.31%
Accuracy measurement from iteration 6 >>> 75.83%
Accuracy measurement from iteration 7 >>> 73.33%
Accuracy measurement from iteration 8 >>> 77.50%
Accuracy measurement from iteration 9 >>> 76.47%
Accuracy measurement from iteration 10 >>> 79.83%
Average model accuracy from all iterations: 77.26% (2.64%)
Total time for model fitting and cross validating: 0:00:25.074383


## 4.a) Alternate Model Three

In [44]:
# encode training and validation datasets
X_train, X_test = encode_data(training_docs, testing_docs, 'freq')
print('The shape of the encoded training dataset:', X_train.shape)
print('The shape of the encoded validation dataset:', X_test.shape)

The shape of the encoded training dataset: (598, 849)
The shape of the encoded validation dataset: (150, 849)


In [45]:
# Initialize the default model and get a baseline result
startTimeModule = datetime.now()
results = list()
iteration = 0
cv = RepeatedKFold(n_splits=N_FOLDS, n_repeats=N_ITERATIONS, random_state=seedNum)
for train_ix, val_ix in cv.split(X_train):
    feature_train, feature_validation = X_train[train_ix], X_train[val_ix]
    target_train, target_validation = y_train[train_ix], y_train[val_ix]
    reset_random(seedNum)
    alternate_model_3 = create_nn_model()
    alternate_model_3.fit(feature_train, target_train, epochs=EPOCH_NUMBER, batch_size=BATCH_SIZE, verbose=0)
    model_metric = alternate_model_3.evaluate(feature_validation, target_validation, verbose=0)[1]
    iteration = iteration + 1
    print('Accuracy measurement from iteration %d >>> %.2f%%' % (iteration, model_metric*100))
    results.append(model_metric)
validation_score = np.mean(results)
validation_variance = np.std(results)
print('Average model accuracy from all iterations: %.2f%% (%.2f%%)' % (validation_score*100, validation_variance*100))
print('Total time for model fitting and cross validating:', (datetime.now() - startTimeModule))

Accuracy measurement from iteration 1 >>> 75.83%
Accuracy measurement from iteration 2 >>> 81.67%
Accuracy measurement from iteration 3 >>> 73.33%
Accuracy measurement from iteration 4 >>> 83.19%
Accuracy measurement from iteration 5 >>> 77.31%
Accuracy measurement from iteration 6 >>> 73.33%
Accuracy measurement from iteration 7 >>> 75.00%
Accuracy measurement from iteration 8 >>> 70.00%
Accuracy measurement from iteration 9 >>> 74.79%
Accuracy measurement from iteration 10 >>> 84.03%
Average model accuracy from all iterations: 76.85% (4.42%)
Total time for model fitting and cross validating: 0:00:24.996135


In [46]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 4 - Evaluate and Optimize Models completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

# Task 5 - Finalize Model and Make Predictions

In [47]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 5 - Finalize Model and Make Predictions has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

In [48]:
# encode training and validation datasets
X_train, X_test = encode_data(training_docs, testing_docs, 'tfidf')
print('The shape of the encoded training dataset:', X_train.shape)
print('The shape of the encoded validation dataset:', X_test.shape)

The shape of the encoded training dataset: (598, 849)
The shape of the encoded validation dataset: (150, 849)


In [49]:
# Create the final model for evaluating the test dataset
reset_random(seedNum)
final_model = create_nn_model()
final_model.fit(X_train, y_train, epochs=EPOCH_NUMBER, batch_size=BATCH_SIZE, verbose=1)

Epoch 1/25
38/38 [==============================] - 0s 2ms/step - loss: 0.6358 - accuracy: 0.6355
Epoch 2/25
38/38 [==============================] - 0s 2ms/step - loss: 0.2082 - accuracy: 0.9465
Epoch 3/25
38/38 [==============================] - 0s 2ms/step - loss: 0.1032 - accuracy: 0.9682
Epoch 4/25
38/38 [==============================] - 0s 2ms/step - loss: 0.0661 - accuracy: 0.9783
Epoch 5/25
38/38 [==============================] - 0s 2ms/step - loss: 0.0497 - accuracy: 0.9900
Epoch 6/25
38/38 [==============================] - 0s 2ms/step - loss: 0.0404 - accuracy: 0.9916
Epoch 7/25
38/38 [==============================] - 0s 2ms/step - loss: 0.0349 - accuracy: 0.9916
Epoch 8/25
38/38 [==============================] - 0s 2ms/step - loss: 0.0308 - accuracy: 0.9916
Epoch 9/25
38/38 [==============================] - 0s 2ms/step - loss: 0.0281 - accuracy: 0.9916
Epoch 10/25
38/38 [==============================] - 0s 2ms/step - loss: 0.0260 - accuracy: 0.9916
Epoch 11/25
38/38 [

In [50]:
# Summarize the final model
final_model.summary()

Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_80 (Dense)             (None, 50)                42500     
_________________________________________________________________
dense_81 (Dense)             (None, 1)                 51        
Total params: 42,551
Trainable params: 42,551
Non-trainable params: 0
_________________________________________________________________


In [51]:
# test_predictions = final_model.predict(X_test, batch_size=default_batch, verbose=1)
test_predictions = (final_model.predict(X_test) > 0.5).astype("int32").ravel()
print('Accuracy Score:', accuracy_score(y_test, test_predictions))
print(confusion_matrix(y_test, test_predictions))
print(classification_report(y_test, test_predictions))

Accuracy Score: 0.6866666666666666
[[50 23]
 [24 53]]
              precision    recall  f1-score   support

           0       0.68      0.68      0.68        73
           1       0.70      0.69      0.69        77

    accuracy                           0.69       150
   macro avg       0.69      0.69      0.69       150
weighted avg       0.69      0.69      0.69       150



In [52]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 5 - Finalize Model and Make Predictions completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

In [53]:
print ('Total time for the script:',(datetime.now() - startTimeScript))

Total time for the script: 0:01:49.988637
